In [1]:
import pandas as pd
import numpy as nm
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, LongType, IntegerType, FloatType
from pyspark.sql.functions import col, column
from pyspark.sql.functions import expr
from pyspark.sql.functions import split
from pyspark.sql import Row
import csv
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, IndexToString, StandardScaler, PCA
from pyspark.ml.classification import DecisionTreeClassifier

In [2]:
from decision_tree_plot.decision_tree_parser import decision_tree_parse
from decision_tree_plot.decision_tree_plot import plot_trees

---

## Initialize a SparkSession, define the csv Schema

In [3]:
ss = SparkSession.builder.appName("Project").getOrCreate()

In [4]:
schema = StructType([ StructField("ID", IntegerType(), False ), \
                        StructField("Case Number", StringType(), False), \
                        StructField("Date", StringType(), False ), \
                        StructField("Block", StringType(), False ), \
                        StructField("IUCR", StringType(), False), \
                        StructField("Primary Type", StringType(), False), \
                        StructField("Description", StringType(), False),\
                        StructField("Location Description", StringType(), False), \
                        StructField("Arrest", StringType(), False), \
                        StructField("Domestic", StringType(), False), \
                        StructField("District", StringType(), False) ,\
                        StructField("Ward", StringType(), False ), \
                        StructField("Community Area", StringType(), False ), \
                        StructField("FBI Code", StringType(), False), \
                        StructField("Year", StringType(), False), \
                        StructField("Latitude", StringType(), False),\
                        StructField("Longitude", StringType(), False)
                           ])

In [5]:
# new_file3.csv is the file generated by using Suvarna's code
data = ss.read.csv("./new_file3.csv",schema=schema,header=True, inferSchema=False)

In [6]:
data = data.drop('Year').drop('Block').drop('Case Number').drop('ID')

---

## Apply PCA Reduction
#### Reason: I tried directly using Lab8's code before. However, I continously got an error saying the data's cardinality is too high and abort the task, which means the dataset's size is too large. To minimize the dataset's size, I need to use PCA reduction first to minimize the size and then use the minimized data to train the model.

`Step 1: Transforms a column of string to a new column of index (type double)`

In [7]:
# This block will create a new column for each indexed column, for example, will create a 'i_date' for indexed 'Date' column
columns_to_index = ["Date", "Primary Type", "Description", "IUCR",
                    "Location Description", "Arrest", "Domestic", "District","Ward","FBI Code","Community Area", 
                    "Latitude", "Longitude"]
label_indexers = {col: StringIndexer(inputCol=col, outputCol=f"i_{col.replace(' ', '_').lower()}").fit(data) for col in columns_to_index}

In [8]:
# This block will transform data
transformed_data = data
for indexer in label_indexers.values():
    transformed_data = indexer.transform(transformed_data)

In [9]:
# data2 will contain the indexed values only, which means they are all integer -> ready for training
data2 = transformed_data.select("i_date","i_iucr","i_primary_type","i_description","i_location_description",\
                    "i_arrest","i_domestic","i_district","i_ward","i_fbi_code","i_community_area","i_latitude","i_longitude")

In [10]:

input_features = ["i_date","i_iucr","i_primary_type","i_description","i_location_description",\
                    "i_domestic","i_district","i_ward","i_fbi_code","i_community_area","i_latitude","i_longitude"]

`Step 2: Create a vector-assembler to combine individual features into a vector in a new column, which called 'features_for_arrest'.`

In [11]:
assembler = VectorAssembler(inputCols=input_features, outputCol="features_for_arrest")

In [12]:
assembled_data = assembler.transform(data2)

`Step 3: Use a StandardScaler to center the data - saved the centered data into a new column called "scaled_features"`

In [13]:
scaler = StandardScaler(inputCol="features_for_arrest", outputCol="scaled_features", withStd=True, withMean=True)
scaler_model = scaler.fit(assembled_data)
scaled_data = scaler_model.transform(assembled_data)

In [14]:
# When k=8, its variance is closest to 0.9 && > 0.9
pca = PCA(k=8,inputCol="scaled_features",outputCol="pcaFeatures")
model = pca.fit(scaled_data)
result = model.transform(scaled_data)
explained_variance = model.explainedVariance
print("Explained Variance: ", sum(explained_variance))

Explained Variance:  0.9360433841638934


`By that, We got the PCA reduction data: result `

---

## Train the model

####  Train the model with the reducted data. I used sklearn instead of PySpark DecisionTree.

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [16]:
# Convert PySpark DataFrame to Pandas DataFrame
# "pcaFeatures" is the hyperparameter that predicts arrest or not
# "i_arrest" is our target variable
pandas_df = result.select("pcaFeatures", "i_arrest").toPandas()

In [31]:
# Extract PCA features and target variable
X = pandas_df["pcaFeatures"].tolist()
y = pandas_df["i_arrest"].tolist()

In [37]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [33]:
# Standardize the data again just in case
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [34]:
# Train the RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train_scaled, y_train)

RandomForestClassifier()

`Above is the model that predicts the value of Arrest`

In [38]:
# Make predictions on the test set
predictions = model.predict(X_test_scaled)

---

## Evaluate the model

In [35]:
# Use test_data to evaluate our model
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: {:.2f}%".format(accuracy * 100))

In [17]:
data.count()

352366

As the original data is too large to run, I decided to start with a sample data.

In [18]:
sample_data = data.limit(1000)

In [19]:
sample_data.show(5)

+--------------------+----+------------+--------------------+--------------------+------+--------+--------+----+--------------+--------+------------+-------------+
|                Date|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|District|Ward|Community Area|FBI Code|    Latitude|    Longitude|
+--------------------+----+------------+--------------------+--------------------+------+--------+--------+----+--------------+--------+------------+-------------+
|05/07/2020 10:24:...|0820|       THEFT|      $500 AND UNDER|           APARTMENT| False|   False|     2.0| 3.0|          35.0|      06|41.830481843|-87.621751752|
|04/16/2020 05:00:...|0460|     BATTERY|              SIMPLE|           APARTMENT|  True|   False|     9.0|11.0|          60.0|     08B|41.836310224|-87.639624112|
|07/01/2020 10:16:...|051A|     ASSAULT|AGGRAVATED - HANDGUN|              STREET|  True|   False|     4.0| 7.0|          46.0|     04A|41.747609555|-87.549179329|
|09/27/2020 11:2

In [20]:
columns_to_index = ["Date", "Primary Type", "Description", "IUCR",
                    "Location Description", "Arrest", "Domestic", "District","Ward","FBI Code","Community Area", 
                    "Latitude", "Longitude"]
label_indexers = {col: StringIndexer(inputCol=col, outputCol=f"indexed_{col.replace(' ', '_').lower()}").fit(sample_data) for col in columns_to_index}

In [21]:
# Apply transformations to the data
transformed_data = sample_data
for indexer in label_indexers.values():
    transformed_data = indexer.transform(transformed_data)

In [22]:
transformed_data.printSchema()

root
 |-- Date: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- indexed_date: double (nullable = false)
 |-- indexed_primary_type: double (nullable = false)
 |-- indexed_description: double (nullable = false)
 |-- indexed_iucr: double (nullable = false)
 |-- indexed_location_description: double (nullable = false)
 |-- indexed_arrest: double (nullable = false)
 |-- indexed_domestic: double (nullable = false)
 |-- indexed_district: double (nullable = false)
 |-- indexed_ward: double (nullable = false)
 |-- indexed_fbi_

My code

In [23]:
transformed_data2 = transformed_data.select("indexed_date","indexed_iucr",\
                                           "indexed_primary_type","indexed_description","indexed_location_description",\
                                           "indexed_arrest","indexed_domestic","indexed_district","indexed_ward",\
                                           "indexed_fbi_code","indexed_community_area","indexed_latitude","indexed_longitude")

In [24]:
transformed_data2.printSchema()

root
 |-- indexed_date: double (nullable = false)
 |-- indexed_iucr: double (nullable = false)
 |-- indexed_primary_type: double (nullable = false)
 |-- indexed_description: double (nullable = false)
 |-- indexed_location_description: double (nullable = false)
 |-- indexed_arrest: double (nullable = false)
 |-- indexed_domestic: double (nullable = false)
 |-- indexed_district: double (nullable = false)
 |-- indexed_ward: double (nullable = false)
 |-- indexed_fbi_code: double (nullable = false)
 |-- indexed_community_area: double (nullable = false)
 |-- indexed_latitude: double (nullable = false)
 |-- indexed_longitude: double (nullable = false)



In [25]:
input_features = ["indexed_date","indexed_iucr","indexed_primary_type","indexed_description","indexed_location_description",\
                  "indexed_domestic","indexed_district","indexed_ward","indexed_fbi_code","indexed_community_area",\
                  "indexed_latitude","indexed_longitude"]

In [26]:
transformed_data2.show(3)

+------------+------------+--------------------+-------------------+----------------------------+--------------+----------------+----------------+------------+----------------+----------------------+----------------+-----------------+
|indexed_date|indexed_iucr|indexed_primary_type|indexed_description|indexed_location_description|indexed_arrest|indexed_domestic|indexed_district|indexed_ward|indexed_fbi_code|indexed_community_area|indexed_latitude|indexed_longitude|
+------------+------------+--------------------+-------------------+----------------------------+--------------+----------------+----------------+------------+----------------+----------------------+----------------+-----------------+
|       200.0|         1.0|                 0.0|                2.0|                         1.0|           0.0|             0.0|             5.0|         4.0|             0.0|                  34.0|           450.0|            216.0|
|       157.0|         6.0|                 1.0|            

In [27]:
assembler = VectorAssembler(inputCols=input_features, outputCol="features_for_arrest")

In [28]:
assembler

VectorAssembler_cfe023d60c00

In [29]:
vectorized_data = assembler.transform(transformed_data2)

In [30]:
vectorized2_data = vectorized_data.select("features_for_arrest",'indexed_arrest')
vectorized2_data.show(5)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/storage/home/fzr5090/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/storage/home/fzr5090/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/swst/apps/anaconda3/2021.05_gcc-8.5.0/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

Decision Tree Learning

In [ ]:
trainingData,testData= vectorized_data.randomSplit([0.75, 0.25], seed=1237)

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline
dt=DecisionTreeClassifier(featuresCol="features_for_arrest", labelCol="indexed_arrest", maxDepth=6, minInstancesPerNode=5, maxBins=10000)

In [ ]:
dt_model = dt.fit(trainingData)

In [ ]:
dt_model